In [49]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

In [50]:

load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")
client=OpenAI(api_key=api_key)


In [52]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [55]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [56]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])

In [63]:
def is_tool_required(user_message):
    if "weather" in user_message.lower() or "temperature" in user_message.lower():
        return True  
    return False  

In [68]:
messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

if is_tool_required(messages[-1]["content"]):
    completion_1 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools 
    )
else:
    completion_1 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

In [69]:

messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})


completion_1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

In [70]:
content=completion_1.choices[0].message.content
print(content)

The current temperature in Paris today is 7.9°C.


In [71]:
messages.append({"role": "user", "content": "Can you recommend a lunch for me?"})


if is_tool_required(messages[-1]["content"]):
    completion_2 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools 
    )
else:
    completion_2 = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

In [72]:
content2=completion_2.choices[0].message.content
print(content2)

Sure! Here are a few lunch ideas:

1. **Quiche Lorraine** - A classic French dish that’s both filling and flavorful.
2. **Croque Monsieur** - A delicious ham and cheese sandwich, often topped with béchamel sauce.
3. **Salad Niçoise** - A fresh salad with tuna, hard-boiled eggs, green beans, and olives, dressed with olive oil.
4. **Baguette Sandwich** - Filled with your choice of meats, cheeses, and vegetables.
5. **Ratatouille** - A hearty vegetable stew, perfect for a light yet satisfying meal.

What type of cuisine are you in the mood for?
